# Artificial Neural Network

### Importing the libraries

In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf

In [ ]:
tf.__version__

'2.12.0'

## Part 1 - Data Preprocessing

### Importing the dataset

In [ ]:
dataset = pd.read_csv('Churn_Modelling.csv')
X = dataset.iloc[:, 3:-1].values
y = dataset.iloc[:, -1].values

In [ ]:
print(X)

[[619 'France' 'Female' ... 1 1 101348.88]
 [608 'Spain' 'Female' ... 0 1 112542.58]
 [502 'France' 'Female' ... 1 0 113931.57]
 ...
 [709 'France' 'Female' ... 0 1 42085.58]
 [772 'Germany' 'Male' ... 1 0 92888.52]
 [792 'France' 'Female' ... 1 0 38190.78]]


### Encoding categorical data

Label Encoding the "Gender" column

In [ ]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
X[:,2] = le.fit_transform(X[:,2])

In [ ]:
print(X)

[[619 'France' 0 ... 1 1 101348.88]
 [608 'Spain' 0 ... 0 1 112542.58]
 [502 'France' 0 ... 1 0 113931.57]
 ...
 [709 'France' 0 ... 0 1 42085.58]
 [772 'Germany' 1 ... 1 0 92888.52]
 [792 'France' 0 ... 1 0 38190.78]]


One Hot Encoding the "Geography" column

In [ ]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
#the type of column transfer
ct = ColumnTransformer(transformers=[('encoder', OneHotEncoder(), [1])], remainder='passthrough')
X =np.array(ct.fit_transform(X))
#dummy var moved to first column

In [ ]:
print(X)

[[1.0 0.0 0.0 ... 1 1 101348.88]
 [0.0 0.0 1.0 ... 0 1 112542.58]
 [1.0 0.0 0.0 ... 1 0 113931.57]
 ...
 [1.0 0.0 0.0 ... 0 1 42085.58]
 [0.0 1.0 0.0 ... 1 0 92888.52]
 [1.0 0.0 0.0 ... 1 0 38190.78]]


### Splitting the dataset into the Training set and Test set

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 1)

### Feature Scaling

In [ ]:
#applied on the training set and test
#this is fundamental to all arti neural nets and applied to all X
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)


## Part 2 - Building the ANN

### Initializing the ANN

In [ ]:
ann = tf.keras.models.Sequential()

### Adding the input layer and the first hidden layer

In [ ]:
#dense class to build fully connected layers at any lvl
ann.add(tf.keras.layers.Dense(units = 6,activation = 'relu'))#units is num of hidden neuronns in a layer but in input layer all neurons will b gen automatically

#activation is relu by def



### Adding the second hidden layer

In [ ]:
ann.add(tf.keras.layers.Dense(units = 6,activation = 'relu'))

### Adding the output layer

In [ ]:
ann.add(tf.keras.layers.Dense(units = 1,activation = 'sigmoid'))

#the brain is done

## Part 3 - Training the ANN

### Compiling the ANN

In [ ]:
# make it more accurate (metrics)
#ann.compile(optimizer = 'adam',loss = 'binary_crossentrophy' ,metrics= ['accuracy'] )   #loss func to calculate the loss to be minimized, optmizer - the ggradients to be used for adjusting weight
# if non-binary - categorical cross enrophy and softmax activation in o/p layer
ann.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

### Training the ANN on the Training set

In [ ]:
ann.fit(X_train, y_train, batch_size = 32, epochs = 100) # bs- we do several predict v actual for faster n bttr
#epoch - num of tym the model goes to n fro thru dataset for optimal weights

Epoch 1/100
250/250 [==============================] - 1s 2ms/step - loss: 0.5710 - accuracy: 0.7558
Epoch 2/100
250/250 [==============================] - 1s 2ms/step - loss: 0.4871 - accuracy: 0.7972
Epoch 3/100
250/250 [==============================] - 1s 2ms/step - loss: 0.4642 - accuracy: 0.7971
Epoch 4/100
250/250 [==============================] - 1s 2ms/step - loss: 0.4510 - accuracy: 0.7981
Epoch 5/100
250/250 [==============================] - 1s 2ms/step - loss: 0.4405 - accuracy: 0.8046
Epoch 6/100
250/250 [==============================] - 1s 2ms/step - loss: 0.4293 - accuracy: 0.8124
Epoch 7/100
250/250 [==============================] - 1s 3ms/step - loss: 0.4176 - accuracy: 0.8159
Epoch 8/100
250/250 [==============================] - 1s 3ms/step - loss: 0.4081 - accuracy: 0.8214
Epoch 9/100
250/250 [==============================] - 1s 3ms/step - loss: 0.3995 - accuracy: 0.8266
Epoch 10/100
250/250 [==============================] - 1s 3ms/step - loss: 0.3929 - accura

## Part 4 - Making the predictions and evaluating the model

### Predicting the result of a single observation

In [ ]:
#must be feature scaled
#enter the hot encoded datas
print(ann.predict(sc.transform([[1,0,0,600,1,40,3,60000,2,1,1,50000 ]])) > 0.5)#always in a matrix

1/1 [==============================] - 0s 98ms/step
[[False]]


**Solution**

### Predicting the Test set results

In [ ]:
y_pred = ann.predict(X_test)
y_pred = (y_pred>0.5)
print(np.concatenate((y_pred.reshape(len(y_pred),1), y_test.reshape(len(y_test),1)),1))

63/63 [==============================] - 0s 2ms/step
[[0 0]
 [0 0]
 [0 0]
 ...
 [0 0]
 [0 0]
 [1 0]]


### Making the Confusion Matrix

In [ ]:
from sklearn.metrics import confusion_matrix, accuracy_score
cm = confusion_matrix(y_test, y_pred)
print(cm)
accuracy_score(y_test, y_pred)

[[1502   83]
 [ 197  218]]


0.86